In [1]:
import copy
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
import tensorflow as tf
import deeppavlov
from deeppavlov.dataset_readers.ontonotes_reader import OntonotesReader

from deeppavlov.models.embedders.glove_embedder import GloVeEmbedder

/home/kostrovskiy/prog/AI/NLP/DeepPavlov/env/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
[nltk_data] Downloading package punkt to
[nltk_data]     /home/kostrovskiy/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/kostrovskiy/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package perluniprops to
[nltk_data]     /home/kostrovskiy/nltk_data...
[nltk_data]   Package perluniprops is already up-to-date!
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     /home/kostrovskiy/nltk_data...
[nltk_data]   Package nonbreaking_prefixes is already up-to-date!
2018-09-11 13:58:04.359 DEBU

In [2]:
from src.fewshot_ner_viz_component.fewshot_ner_binary_classifier import FewshotNerBinaryClassifier, CompositeEmbedder
from src.fewshot_ner_viz_component.data_processing import *

### Configuration parameters

In [3]:
NE_TYPE = 'PERSON'
N_EXAMPLES = list(range(1, 11))
N_EXAMPLES = list(range(5, 11))
# N_EXAMPLES.extend([20,30,40,50,60,70,80,90,100])
N_EXAMPLES = list(range(10, 101, 10))
N_EXAMPLES = list(range(10, 101, 10)) + list(range(200, 1001, 100))
N_TEST_SENTENCES = 100
N_SAMPLES = 10
# MODELS = ['ne_centroid', 'ne_nearest', 'weighted_kNN', 'centroid_kNN', 'svm']
MODELS = ['ne_centroid', 'ne_nearest', 'weighted_kNN', 'centroid_kNN', 'svm']
MODELS = ['ne_centroid', 'weighted_kNN', 'centroid_kNN', 'svm']
MODELS = ['svm', 'crf', 'logreg']
MODELS_COMPOSE = MODELS
MODEL_PARAMS = {'ne_centroid': {'sim_type': 'cosine'},
               'ne_nearest': {'sim_type': 'cosine'},
               'weighted_kNN': {'k': 1, 'metric': 'cosine', 'use_class_weights': False, 'use_sim_weights': True},
               'centroid_kNN': {'k': 10, 'metric': 'dot_prod', 'use_class_weights': False},
               'svm': {},
               'crf': {'c2': 0, 'c1': 0}}

In [4]:
dataset_orig = read_data()

Num of train sentences: 75187
Num of valid sentences: 9603
Num of test sentences: 9479
[(['Actions', 'had', 'to', 'be', 'taken', 'to', 'break', 'through', 'the', 'blockade', '.'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']), (['On', 'a', 'night', 'in', 'late', 'July', '1940', ',', 'the', 'atmosphere', 'in', 'Zhuanbi', 'Village', 'in', 'Shaanxi', 'was', 'unusual', '.'], ['O', 'B-DATE', 'I-DATE', 'I-DATE', 'I-DATE', 'I-DATE', 'I-DATE', 'O', 'O', 'O', 'O', 'B-GPE', 'I-GPE', 'O', 'B-GPE', 'O', 'O', 'O']), (['Villager', 'Xiao', 'Jianghe', 'has', 'a', 'vivid', 'memory', 'of', 'this', 'piece', 'of', 'history', '.'], ['O', 'B-PERSON', 'I-PERSON', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']), (['On', 'that', 'dark', 'night', ',', 'everyone', 'was', 'sleeping', 'when', 'human', 'voices', 'and', 'neighing', 'horses', 'were', 'heard', 'within', 'the', 'village', '.'], ['O', 'B-TIME', 'I-TIME', 'I-TIME', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', '

In [5]:
# embedder = CompositeEmbedder(use_elmo=True, elmo_scale=1, cap_scale=1, use_cap_feat=True, use_glove=True, elmo_params={'trainable_cells': True, 'restore_path': "./tmp/elmo.ckpt"})
embedder = CompositeEmbedder(use_elmo=True, elmo_scale=1, cap_scale=1, use_cap_feat=True, use_glove=True)
# embedder = CompositeEmbedder(use_elmo=False, elmo_scale=1, cap_scale=1, use_cap_feat=False, use_glove=False, use_fastText=True)

INFO:tensorflow:Using /tmp/tfhub_modules to cache modules.


2018-09-11 13:58:20.543 INFO in 'tensorflow'['tf_logging'] at line 160: Using /tmp/tfhub_modules to cache modules.


INFO:tensorflow:Initialize variable module/aggregation/scaling:0 from checkpoint b'/tmp/tfhub_modules/9bb74bc86f9caffc8c47dd7b33ec4bb354d9602d/variables/variables' with aggregation/scaling


2018-09-11 13:58:21.497 INFO in 'tensorflow'['tf_logging'] at line 116: Initialize variable module/aggregation/scaling:0 from checkpoint b'/tmp/tfhub_modules/9bb74bc86f9caffc8c47dd7b33ec4bb354d9602d/variables/variables' with aggregation/scaling


INFO:tensorflow:Initialize variable module/aggregation/weights:0 from checkpoint b'/tmp/tfhub_modules/9bb74bc86f9caffc8c47dd7b33ec4bb354d9602d/variables/variables' with aggregation/weights


...

2018-09-11 13:58:21.762 INFO in 'tensorflow'['tf_logging'] at line 116: Initialize variable module/bilm/RNN_1/RNN/MultiRNNCell/Cell1/rnn/lstm_cell/kernel:0 from checkpoint b'/tmp/tfhub_modules/9bb74bc86f9caffc8c47dd7b33ec4bb354d9602d/variables/variables' with bilm/RNN_1/RNN/MultiRNNCell/Cell1/rnn/lstm_cell/kernel


INFO:tensorflow:Initialize variable module/bilm/RNN_1/RNN/MultiRNNCell/Cell1/rnn/lstm_cell/projection/kernel:0 from checkpoint b'/tmp/tfhub_modules/9bb74bc86f9caffc8c47dd7b33ec4bb354d9602d/variables/variables' with bilm/RNN_1/RNN/MultiRNNCell/Cell1/rnn/lstm_cell/projection/kernel


2018-09-11 13:58:21.772 INFO in 'tensorflow'['tf_logging'] at line 116: Initialize variable module/bilm/RNN_1/RNN/MultiRNNCell/Cell1/rnn/lstm_cell/projection/kernel:0 from checkpoint b'/tmp/tfhub_modules/9bb74bc86f9caffc8c47dd7b33ec4bb354d9602d/variables/variables' with bilm/RNN_1/RNN/MultiRNNCell/Cell1/rnn/lstm_cell/projection/kernel


INFO:tensorflow:Initialize variable module/bilm/char_embed:0 from checkpoint b'/tmp/tfhub_modules/9bb74bc86f9caffc8c47dd7b33ec4bb354d9602d/variables/variables' with bilm/char_embed


2018-09-11 13:58:21.781 INFO in 'tensorflow'['tf_logging'] at line 116: Initialize variable module/bilm/char_embed:0 from checkpoint b'/tmp/tfhub_modules/9bb74bc86f9caffc8c47dd7b33ec4bb354d9602d/variables/variables' with bilm/char_embed
2018-09-11 13:58:28.883 INFO in 'deeppavlov.models.embedders.glove_embedder'['glove_embedder'] at line 60: [loading embeddings from `/home/kostrovskiy/prog/AI/NLP/DeepPavlov/download/embeddings/glove.6B/glove.6B.100d.txt`]
2018-09-11 13:58:28.884 INFO in 'gensim.models.keyedvectors'['keyedvectors'] at line 204: loading projection weights from /home/kostrovskiy/prog/AI/NLP/DeepPavlov/download/embeddings/glove.6B/glove.6B.100d.txt
2018-09-11 13:58:28.884 DEBUG in 'smart_open.smart_open_lib'['smart_open_lib'] at line 176: {'kw': {}, 'mode': 'rb', 'uri': '/home/kostrovskiy/prog/AI/NLP/DeepPavlov/download/embeddings/glove.6B/glove.6B.100d.txt'}
2018-09-11 13:59:03.80 INFO in 'gensim.models.keyedvectors'['keyedvectors'] at line 266: loaded (400000, 100) matri

In [ ]:
from collections import defaultdict
f1_scores = {}
print('NE type: {}'.format(NE_TYPE))
dataset = filter_dataset_by_ne_types(dataset_orig, NE_TYPE, preserveBIO=True)
for n_ex in N_EXAMPLES:
    print('++++++++ Number of example sentences: {} ++++++++'.format(n_ex))
    f1_scores[n_ex] = {}
    for model in MODELS:
        f1_scores[n_ex][model] = {'values':[], 'values_chunk':[]}
    for idx_sample in range(N_SAMPLES):
        print('Sample {}'.format(idx_sample+1))
        tokens_train,tags_train = get_data_sample(dataset['train'], n_ex)
        tags_train = removeBIOFromTags(tags_train)
        tokens_test,tags_test = get_data_sample(dataset['test'], N_TEST_SENTENCES)
        tags_test_bio = tags_test
        tags_test_bio_flat = flatten_list(tags_test_bio)
        tags_test = removeBIOFromTags(tags_test)
        y_test = np.array([1 if t == 'T' else 0 for seq in tags_test for t in seq])
        ner_bin_clf = FewshotNerBinaryClassifier(embedder, use_crf=True, use_logreg=True, model_params=MODEL_PARAMS)
        ner_bin_clf.train_on_batch(tokens_train, tags_train)
        results = ner_bin_clf.predict(tokens_test, MODELS, MODEL_PARAMS)
        pred_avg = np.zeros(y_test.size)
        n_models_avg = 0
        for model in MODELS:
            pred = results[model]['pred']
#             if model in MODELS_COMPOSE:   
#                 pred_avg += pred
#                 n_models_avg += 1
            f1 = f1_score(y_test, pred)
            f1_scores[n_ex][model]['values'].append(f1)
            # chunk-wise f1
            pred_tags_bio = predToTags(pred, accountBIO=True)
            f1_chunk = precision_recall_f1_chunks(tags_test_bio_flat, pred_tags_bio, print_results=False)['__total__']['f1']/100
            f1_scores[n_ex][model]['values_chunk'].append(f1_chunk)
#         pred_avg /= n_models_avg
#         pred_avg[pred_avg >= 0.5] = 1
#         pred_avg[pred_avg < 0.5] = 0
#         f1 = f1_score(y_test, pred_avg)
#         pred_tags_bio_avg = predToTags(pred_avg, accountBIO=True)
#         f1_chunk = precision_recall_f1_chunks(tags_test_bio_flat, pred_tags_bio_avg, print_results=False)['__total__']['f1']/100
#         f1_scores[n_ex]['compose']['values'].append(f1)
#         f1_scores[n_ex]['compose']['values_chunk'].append(f1_chunk)
        
        cur_f1_scores_dict = {k: f1_scores[n_ex][k]['values'][-1] for k in f1_scores[n_ex].keys()}
        print('F1 scores: {}'.format(cur_f1_scores_dict))
        
        del ner_bin_clf
    for model in MODELS:
        f1_scores[n_ex][model]['mean'] = np.mean(f1_scores[n_ex][model]['values'])
        f1_scores[n_ex][model]['std'] = np.std(f1_scores[n_ex][model]['values'])
        f1_scores[n_ex][model]['mean_chunk'] = np.mean(f1_scores[n_ex][model]['values_chunk'])
        f1_scores[n_ex][model]['std_chunk'] = np.std(f1_scores[n_ex][model]['values_chunk'])
        print('{} F1: mean = {} std = {}'.format(model, f1_scores[n_ex][model]['mean'], f1_scores[n_ex][model]['std']))
        print('{} F1: mean_chunk = {} std_chunk = {}'.format(model, f1_scores[n_ex][model]['mean_chunk'], f1_scores[n_ex][model]['std_chunk']))

NE type: PERSON
Num of train sentences: 12195
Num of valid sentences: 1553
Num of test sentences: 1573
++++++++ Number of example sentences: 10 ++++++++
Sample 1
{'algorithm': 'lbfgs', 'c1': 0, 'c2': 0}
Train
# ne: 22, # tokens: 238
n_samples: 238
SVM classifier model
CRF classifier model
LogReg classifier model
F1 scores: {'svm': 0.8625336927223719, 'crf': 0.8362573099415205, 'logreg': 0.8402366863905325}
svm F1: mean = 0.8625336927223719 std = 0.0
svm F1: mean_chunk = 0.7449392712550608 std_chunk = 0.0
crf F1: mean = 0.8362573099415205 std = 0.0
crf F1: mean_chunk = 0.7407407407407406 std_chunk = 0.0
logreg F1: mean = 0.8402366863905325 std = 0.0
logreg F1: mean_chunk = 0.6844444444444443 std_chunk = 0.0
++++++++ Number of example sentences: 20 ++++++++
Sample 1
{'algorithm': 'lbfgs', 'c1': 0, 'c2': 0}
Train
# ne: 35, # tokens: 468
n_samples: 468
SVM classifier model
CRF classifier model
LogReg classifier model
F1 scores: {'svm': 0.8658008658008659, 'crf': 0.9002320185614849, 'logreg

In [ ]:
f1_by_model = {}
f1_chunk_by_model = {}
for n_ex, results in f1_scores.items():
    for model in results.keys():
        if not f1_by_model.get(model):  
            f1_by_model[model] = []
            f1_chunk_by_model[model] = []
        f1_by_model[model].append(results[model]['mean'])
        f1_chunk_by_model[model].append(results[model]['mean_chunk'])

In [ ]:
markers = ['^', 'o', 's', 'x', 'v', '*', '<', '>']
# markers = ['^', 'v', 'o', 's']
fig = plt.figure()
i = 0
for model, f1_scores_list in f1_by_model.items():
    ax = plt.semilogx(N_EXAMPLES, f1_scores_list, marker=markers[i], label=model)
    i += 1
plt.grid()
plt.xticks(N_EXAMPLES)
plt.yticks(np.arange(0.8,0.95,0.1))
plt.xlabel('# examples')
plt.ylabel('F1 score')
plt.legend()
plt.title('Average F1 scores')

### Token-wise vs chunk-wise F1 scores

In [ ]:
fig = plt.figure()
plt.plot(N_EXAMPLES, f1_by_model['svm'], marker='s', c='b', label='token-wise')
plt.plot(N_EXAMPLES, f1_chunk_by_model['svm'], marker='o', c='r', label='chunk-wise')
plt.grid()
plt.xticks(N_EXAMPLES)
plt.xlabel('# examples')
plt.ylabel('F1 score')
plt.legend()
plt.title('Average F1 scores for SVM: token-wise vs chunk-wise')

In [ ]:
fig = plt.figure()
plt.plot(N_EXAMPLES, f1_by_model['weighted_kNN'], marker='s', c='b', label='token-wise')
plt.plot(N_EXAMPLES, f1_chunk_by_model['weighted_kNN'], marker='o', c='r', label='chunk-wise')
plt.grid()
plt.xticks(N_EXAMPLES)
plt.xlabel('# examples')
plt.ylabel('F1 score')
plt.legend()
plt.title('Average F1 scores for kNN: token-wise vs chunk-wise')